In [151]:
service = dict([('title', 'S1 data selection'),
                ('abstract', 'Sentinel-1 data selection'),
                ('id', 's1_crop')])

In [ ]:
base_endpoint = dict([('id', 'base_endpoint'),
              ('value', 'https://catalog.terradue.com/sentinel1/series/SLC_GRDH/search'),
              ('title', 'catalogue base URL'),
              ('abstract', 'base URL of the catalogue')])

In [ ]:
start = dict([('id', 'start'),
              ('value', '2018-06-08T00:00:00Z'),
              ('title', 'start date for dataset selection'),
              ('abstract', 'start date for dataset selection')])

In [157]:
stop = dict([('id', 'stop'),
            ('value', '2018-06-08T23:59:59Z'),
            ('title', 'stop date for dataset selection'),
            ('abstract', 'stop date for dataset selection')])

In [ ]:
search_type = dict([('id', 'time_filter_type'),
            ('value', 'date'),
            ('title', 'search time filter type'),
            ('abstract', 'search time filter type for dataset selection (date|update)')])

In [158]:
cat_index = dict([('id', 'cat_index'),
              ('value', 'ecop-cnr-issia'),
              ('title', 'publishing catalog index'),
              ('abstract', 'publishing catalog index')])

In [159]:
config_url = dict([('id', 'config_url'),
                   ('value', 'https://store.terradue.com/ec-ecopotential/config/config.ini'),
                   ('title', 'FTP configuration file URL'),
                   ('abstract', 'FTP configuration file URL')])

In [160]:
username = dict([('id', 'username'),
                 ('value', 'ecop-cnr-issia'),
                 ('title', 'username for DA access'),
                 ('abstract', 'username for DA access')])

In [161]:
api_key  = dict([('id', 'api_key'),
                 ('value', 'AKCp5aUa1sSVSDdky5odFrKbwpyKzrBuZjWvrWbUigaPVNPWL3H6uiLSgEjFWLVdnUz45Ky7x'),
                 ('title', 'apikey for DA access'),
                 ('abstract', 'apikey for DA access')])

In [164]:
data_path = '/workspace/data'

In [162]:
input_reference = "donana"

In [165]:
#input_identifier = 'S1A_IW_SLC__1SDV_20180503T182621_20180503T182648_021744_025843_8690'
input_identifier = 'S1A_IW_SLC__1SDV_20180608T182623_20180608T182650_022269_0268EB_AB32'

In [166]:
#input_identifier_wkt = 'POLYGON((-8.535965 37.180996,-5.694044 37.580723,-5.365185 35.960056,-8.145734 35.558769,-8.535965 37.180996))'
input_identifier_wkt = 'POLYGON((-8.536756 37.18034,-5.697594 37.579758,-5.368666 35.959213,-8.146519 35.558239,-8.536756 37.18034))'

In [ ]:
input_pa_name = 'Donana'

In [163]:
input_ftp_server_address = 'sftp://frontend.recas.ba.infn.it/lustre/ecopotential/incoming/PAs'

In [170]:
import os
import shutil
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import snappy
import dateutil.parser as parser
from datetime import datetime

from os.path import basename
import osr
import warnings
warnings.filterwarnings("ignore")
from contextlib import closing
from zipfile import ZipFile, ZIP_DEFLATED
import zipfile
import sys

sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())
import ISOMetadata

from shapely.wkt import loads

In [171]:
def unzip(source_filename, dest_dir):
    with zipfile.ZipFile(source_filename) as zf:
        zf.extractall(dest_dir)

In [173]:
unzip('%s/%s.zip' % (data_path,input_identifier), '%s/%s' % (data_path,input_identifier))

In [174]:
s1prd = "%s/%s/%s.SAFE/manifest.safe" % (data_path, input_identifier, input_identifier)
product = snappy.ProductIO.readProduct(s1prd)

width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()
prod_type = product.getProductType()
name = product.getName()
description = product.getDescription()
band_names = product.getBandNames()


In [175]:
start_Date = parser.parse(product.getStartTime().toString()).date()
start_time = parser.parse(product.getStartTime().toString()).time()
end_Date = parser.parse(product.getEndTime().toString()).date()
end_time = parser.parse(product.getEndTime().toString()).time()
now = datetime.now()

start_date = '%sT%sZ' %(start_Date,start_time)
end_date = '%sT%sZ' %(end_Date,end_time)
metadata_date = now.strftime("%Y-%m-%d")
creation_date = start_date


In [176]:
#date_path = '%s/%02d/%02d' % (product_date.year, product_date.month, product_date.day)
date_path = '%s/%02d/%02d' % (start_Date.year, start_Date.month, start_Date.day)
    
#middle_path = '%s/EO_Data/Sentinel1/Raw'%pa_name['value']
middle_path = '%s/EO_Data/Sentinel1/Raw'%input_pa_name
        
ftp_remote_path = os.path.join(input_ftp_server_address, date_path)
output_name = name
download_URL = '%s/%s' %(ftp_remote_path, output_name)

In [177]:
def set_dates(start_date,end_date):
    
    iso_metadata.set_start_date(start_date)
    iso_metadata.set_end_date(end_date)

In [178]:
def set_info(short_name, title, abstract):
    
    iso_metadata.set_identifier(short_name)
    iso_metadata.set_title(title)
    iso_metadata.set_abstract(abstract)

In [179]:
def set_geo(wkt,height,width):
       
    iso_metadata.set_col_size(height)
    iso_metadata.set_row_size(width)
    
    #POLYGON((-26.4804 33.505,-26.7143 32.3182,-24.1248 32.0496,-23.8554 33.2363,-26.4804 33.505))
    polygon = loads(wkt)
    
    min_x,min_y,max_x,max_y = polygon.bounds
    
    lr_x,lry,ul_x,ul_y = polygon.bounds
    
    nw_corner = '%s %s' % (str(min_x), str(max_y))

    iso_metadata.set_nw_corner(nw_corner)

    se_corner = '%s %s' % (str(max_x), str(min_y))

    iso_metadata.set_se_corner(se_corner)
    
    iso_metadata.set_min_lon(str(min_x))

    iso_metadata.set_min_lat(str(min_y))
    
    iso_metadata.set_max_lon(str(max_x))

    iso_metadata.set_max_lat(str(max_y))


In [180]:
iso_metadata = ISOMetadata.ISOMetadata()
iso_metadata.set_organisation('CNR')
iso_metadata.set_contact('info@terradue.com')
iso_metadata.set_date(metadata_date)
iso_metadata.set_creation_date(creation_date)
iso_metadata.set_data_format('SAFE')
iso_metadata.set_data_type('CInt16')
#iso_metadata.set_pa(pa_name['value'])
iso_metadata.set_pa(input_pa_name)
iso_metadata.set_responsible_party('CNR')
iso_metadata.set_onlineResource(download_URL)
#iso_metadata.set_pa('Area covering %s PA' %pa_name['value'])
iso_metadata.set_pa('Area covering %s PA' %input_pa_name)

### Writing General XML

In [181]:
set_dates(start_date,end_date)
    
short_name = name
title = 'Sentinel-1 SAR Standard L1 %s Product %s ' % (prod_type, name)
#abstract = 'Sentinel-1 SAR Standard L1 %s Product covering %s ' % (prod_type, pa_name['value'])
abstract = 'Sentinel-1 SAR Standard L1 %s Product covering %s ' % (prod_type, input_pa_name)
        
set_info(short_name, title, abstract)
set_geo(input_identifier_wkt, str(height).decode("utf-8"), str(width).decode("utf-8"))

iso_metadata.write(name + '.xml')


### Moving the original zip archive into the runtime dir

In [ ]:
shutil.move('%s/%s.zip' % (data_path,input_identifier),'./%s.zip' %input_identifier)